In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

In [28]:
import dataset_utils
import importlib

In [14]:
importlib.reload(dataset_utils)

<module 'dataset_utils' from 'd:\\University\\P4P\\P4P-mmWave\\MMWave_Radar_Human_Tracking_and_Fall_detection\\dataset_utils.py'>

In [32]:
DATA_DIR = "2_class_data"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: sitting
processing class: standing


In [33]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [34]:
print(train_points)

[[[ 3.616e-01  3.816e-01  8.257e-01  3.757e-01  3.100e+02]
  [ 0.000e+00  1.298e-01  7.607e-01  5.010e-01  2.420e+02]
  [ 2.593e-01  1.755e-01  9.385e-01  3.757e-01  2.690e+02]
  ...
  [ 4.907e-01  7.251e-01  4.133e-01  1.252e-01  1.230e+02]
  [ 2.433e-01  1.755e-01  9.150e-01  3.757e-01  2.690e+02]
  [ 3.616e-01  6.870e-01  7.090e-01  1.252e-01  2.290e+02]]

 [[ 8.418e-01  2.590e-01  4.167e-01  3.757e-01  1.800e+02]
  [ 4.050e-01  3.235e-01  0.000e+00 -2.505e-01  1.890e+02]
  [ 7.007e-01  2.302e-01  5.596e-01  2.505e-01  1.170e+02]
  ...
  [ 6.113e-01  4.634e-01  1.061e-01 -2.505e-01  1.210e+02]
  [ 3.096e-01  4.019e-01  4.175e-01 -2.505e-01  1.510e+02]
  [ 2.433e-01  3.445e-01  6.660e-01 -3.757e-01  2.440e+02]]

 [[ 5.010e-01  5.098e-01  7.041e-01  3.757e-01  4.190e+02]
  [ 3.115e-01  7.573e-01  3.987e-01 -5.010e-01  3.280e+02]
  [ 6.235e-01  5.078e-01  3.738e-01 -3.757e-01  3.480e+02]
  ...
  [ 1.000e+00  3.999e-01  9.082e-01  3.757e-01  3.970e+02]
  [ 2.888e-01  2.063e-01  8.706e-0

In [35]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [36]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [37]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [38]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [39]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

#model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape))
#model.add(layers.MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))
#model.add(BatchNormalization())

# # 2. LSTM layer
# model.add(layers.LSTM(50, return_sequences=True))
# model.add(layers.LSTM(50))

# 3. Dense layers
#model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(layers.Dense(1, activation='linear'))  # Output layer (you can adjust the output size and activation)
model.add(layers.Dense(1, activation='softmax'))


In [41]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, None, 64)          1024      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, None, 64)          0         
                                                                 
 batch_normalization_3 (Bat  (None, None, 64)          256       
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, None, 64)          4160      
                                                                 
 dense_3 (Dense)             (None, None, 1)          

In [42]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, batch_size=16, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
70/70 [==============================] - 2s 7ms/step - loss: 0.4973 - mae: 0.4973 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 2/30
70/70 [==============================] - 0s 4ms/step - loss: 0.4973 - mae: 0.4973 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 3/30
70/70 [==============================] - 0s 4ms/step - loss: 0.4973 - mae: 0.4973 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 4/30
70/70 [==============================] - 0s 4ms/step - loss: 0.4973 - mae: 0.4973 - val_loss: 0.4964 - val_mae: 0.4964


In [43]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

9/9 [==============================] - 0s 3ms/step - loss: 0.4964 - mae: 0.4964


In [44]:
print(test_acc)

0.49642857909202576


In [29]:
# pickle.dump(model, open("model.pkl", "wb"))

In [45]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# Flatten the output
model.add(layers.Flatten())

# Summary of the CNN model
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, None, 64)          1024      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, None, 64)          0         
                                                                 
 batch_normalization_4 (Bat  (None, None, 64)          256       
 chNormalization)                                                
                                                                 
 conv1d_5 (Conv1D)           (None, None, 64)          12352     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, None, 64)         

In [46]:
import numpy as np

# Assuming sequence_length = 100 for this example
sequence_length = 100
X = np.random.random((10, sequence_length, 5))  # Example data

# Extract features using CNN
X_Train_Features = model.predict(train_points)
X_Test_Features = model.predict(test_points)

9/9 [==============================] - 0s 2ms/step


In [47]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

svm_classifer = svm.SVC(kernel='linear')
svm_classifer.fit(X_Train_Features, train_labels)

y_pred = svm_classifer.predict(X_Test_Features)

accuracy = accuracy_score(test_labels, y_pred)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 87.14


In [48]:
print(len(X_Train_Features))
print(len(X_Test_Features))
print(len(train_labels))
print(len(test_labels))

1114
280
1114
280
